# NLP Ngram language model lab SOLUTIONS

In this lab you will do 4 exercises building and evaluating ngram language models of the following types:
1. A Maximum Likelihood Estimation (MLE) unigram model
2. A bigram model with add-one Laplace smoothing
3. A bigram model with general additive/Lidstone (add-k) smoothing
4. Ngram models with an advanced interpolation technique, Kneser-Ney snoothing (the methods are provided)

Before you start the exercises, make sure you run and understand the examples first. Then complete the exercises using the following 3 files with line-separated text to train the bigger language models on:
* training data -- "switchboard_lm_training.txt"
* heldout/development data -- "switchboard_lm_heldout.txt"
* test data -- "switchboard_lm_test.txt"

In [1]:
from collections import Counter
from math import log
LOG_BASE = 2 # all the way through here we will use log base 2

In [2]:
# Some useful methods for all exercises:
def glue_tokens(tokens, order):
    """A useful way of glueing tokens together for
    Kneser Ney smoothing and other smoothing methods
    
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    return '{0}@{1}'.format(order,' '.join(tokens))

def unglue_tokens(tokenstring, order):
    """Ungluing tokens glued by the glue_tokens method"""
    if order == 1:
        return [tokenstring.split("@")[1].replace(" ","")]
    return tokenstring.split("@")[1].split(" ")

def tokenize_sentence(sentence, order):
    """Returns a list of tokens with the correct numbers of initial
    and end tags (this is meant ot be used with a non-backoff model!!!)
    
    :sentence: a string of text
    :param: order is the order of the language model
        (1 = unigram, 2 = bigram, 3 =trigram etc.)
    """
    tokens = sentence.split()
    tokens = ['<s>'] * (order-1) + tokens + ['</s>']
    return tokens

# Examples

In [3]:
# Set of sentences (corpus) from the example in the lecture slides
sentences = [
            "I am Sam",
            "Sam I am",
            "I do not like green eggs and ham"
            ]

## Example 1. Build a unigram MLE language model from a simple corpus.
An MLE unigram model will tell you how likely a word is to occur, estimated from the function of counts:

C(w_i)/N

where C(w_i) is the number of times the word at position i occurred in the training corpus, and N is the sum of the counts of all words, or, to put it another way, the length of the training corpus.

Notice the tokenization method adds a `</s>` at the end but no `<s>` is needed at the beginning of each sentence
    because unigrams do not have a context word (we are only concerned with the frequency of single words).

In [4]:
unigrams = Counter()
for sent in sentences:
    words = tokenize_sentence(sent, 1)
    print("tokenized", words)
    for w in words:
        unigrams[w] +=1
unigram_total = sum(unigrams.values()) # to get the denominator for unigram probabilities
print(len(unigrams), "different unigrams observed")
print("unigram total", unigram_total)

tokenized ['I', 'am', 'Sam', '</s>']
tokenized ['Sam', 'I', 'am', '</s>']
tokenized ['I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>']
11 different unigrams observed
unigram total 17


To evaluate the model, we will measure the model's perplexity of those same training sentences. Note in normal practice you would want to do this on different data (as you will do below).

Perplexity is equal to 2 to the power of the cross entropy where cross entropy is the negative sum of all log probabilities from the model normalized by the length of the corpus N.

Measure the cross entropy and perplexity on each sentence too.

In [5]:
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s 
for sent in sentences:
    # get the unigram model based probability of each sentence
    words = tokenize_sentence(sent, 1) # tokenize sentence with the order 1 as the parameter
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for w in words:
        prob = unigrams[w]/unigram_total
        logprob = log(prob, LOG_BASE)  # the log of the prob to base 2
        s += -log(prob, LOG_BASE) # add the neg log prob to s
        sent_s += -log(prob, LOG_BASE)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N  # cross entropy total neg log probs/length for this sentence
    sent_perplexity = LOG_BASE ** sent_cross_entropy # perplexity for the sentence 2 to the cross-entropy
    print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N # cross entropy of corpus total neg log probs/length
perplexity = 2 ** cross_entropy  # perplexity for the corpus 2 to the cross-entropy
print("unigram corpus cross entropy", cross_entropy)
print("unigram corpus perplexity", perplexity)

['I', 'am', 'Sam', '</s>'] cross entropy: 2.7949815908897615 perplexity: 6.940220937885672
['Sam', 'I', 'am', '</s>'] cross entropy: 2.7949815908897615 perplexity: 6.940220937885672
['I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>'] cross entropy: 3.7352489522011934 perplexity: 13.317477627933627
unigram corpus cross entropy 3.2927701939369913
unigram corpus perplexity 9.799921507045037


## Example 2. Build a bigram MLE language model from the same corpus
A MLE (unsmoothed) bigram model will tell you how likely a word is to occur given the previous word, estimated from the function of counts:

C(w_i-1, w_i)/C(w_i-1)

where for any pairs of contiguous words w_i-1, w_i, C(w_i-1, w_i) is the number of times the word at position i follows the word at position i-1 in the training corpus, and C(w_i-1) is the number of times the word at position i-1 occurs in the corpus. E.g. for the bigram probability of 'john likes', C(w_i-1, w_i) is the number of times 'john likes' occurs, and C(w_i-1) is how many times 'john' occurs.

Notice the tokenization method adds a `</s>` at the end and also one `<s>` for padding at the beginning as we want to count the number of times the word at the beginning of each sentence begins a sentence.

In [6]:
# First get the counts from the training corpus for bigrams without smoothing
bigrams = Counter() # a counter for how many times a given bigram sequence w_i-1,w_i occurs
bigram_context = Counter() # a counter for how many times each word is used as a context word w_i-1 (so will include the start symbol)
order = 2 # order (i.e. n) of the language model- bigram n=2
for s in sentences:
    words = tokenize_sentence(s, order)  # tokenize sentence with the order 2 as the parameter
    for i in range(order - 1, len(words)):
        context = words[i-order+1:i]
        target = words[i]
        ngram = context + [target]
        bigrams[glue_tokens(ngram, order)] +=1
        bigram_context[glue_tokens(context, 1)] += 1
print(len(bigrams.keys()), "different bigrams")
print(len(bigram_context.keys()), "different bigram contexts (and unigrams) observed")

15 different bigrams
11 different bigram contexts (and unigrams) observed


In [8]:
# a handy function to calculate the probability of an bigram from the counts
def prob_bigram_MLE(ngram):
    """A simple function to compute the 
    MLE probability estimation based on the counts.
    Follows the equation:
    C(w_i-1, w_i)/C(w_i-1)
    
    Dictionaries bigrams and bigram_context are global variables.

    """
    numerator = bigrams[glue_tokens(ngram, 2)]
    denominator = bigram_context[glue_tokens(ngram[:1], 1)]
    prob = numerator / denominator
    return prob


if False: # optional- check to see if continuation distributions sum to 1
    # check if each bigram continuation distribution sums to one
    # look at the distributions of possible continuations after each word
    for context, v in bigram_context.items():
        context = unglue_tokens(context, 1)
        print("%% context", context)
        check_ngram_total_sums_to_1 = 0
        # for a given context the continuation probabilities 
        # over the whole vocab should sum to 1
        for u in unigrams.keys():
            ngram = context + [u]
            prob = prob_bigram_MLE(ngram)
            print(ngram, prob)
            check_ngram_total_sums_to_1 += prob
        print("sums to 1?", check_ngram_total_sums_to_1)

In [10]:
# Check the estimates for the lecture examples:
# p(I|<s>)
# p(Sam|<s>)
# p(am|I)
# p(</s>|Sam)
# p(Sam|am)
# p(do|I)

print(prob_bigram_MLE(['<s>','I']))
print(prob_bigram_MLE(['<s>', 'Sam']))
print(prob_bigram_MLE(['I', 'am']))
print(prob_bigram_MLE(['Sam', '</s>']))
print(prob_bigram_MLE(['am', 'Sam']))
print(prob_bigram_MLE(['I', 'do']))

0.6666666666666666
0.3333333333333333
0.6666666666666666
0.5
0.5
0.3333333333333333


To evaluate the model, as in the unigram case above we will measure the model's perplexity of those same training sentences.

Notice that even with this small corpus the bigram perplexity is significantly lower than the unigram perplexity.

In [11]:
s = 0  # total neg log prob mass for cross entropy
N = 0 # total number of words for normalizing s
for sent in sentences:
    words = tokenize_sentence(sent, order)
    sent_s = 0  # recording non-normalized entropy for this sentence
    sent_N = 0  # total number of words in this sentence (for normalization)
    for i in range(order - 1, len(words)):
        context = words[i-order+1:i]
        target = words[i]
        ngram = context + [target]
        prob = prob_bigram_MLE(ngram)
        s += -log(prob, LOG_BASE) # add the neg log prob to s
        sent_s += -log(prob, LOG_BASE)  # add the neg log prob to sent_s
        N += 1 # increment the number of total words
        sent_N += 1 # increment the number of total words in this sentence
    sent_cross_entropy = sent_s/sent_N  # cross entropy total neg log probs/length for this sentence
    sent_perplexity = LOG_BASE ** sent_cross_entropy # perplexity for the sentence 2 to the cross-entropy
    print(words, "cross entropy:", sent_cross_entropy, "perplexity:", sent_perplexity)
cross_entropy = s/N # cross entropy of corpus total neg log probs/length
perplexity = 2 ** cross_entropy  # perplexity for the corpus 2 to the cross-entropy
print("bigram corpus cross entropy", cross_entropy)
print("bigram corpus perplexity", perplexity)

['<s>', 'I', 'am', 'Sam', '</s>'] cross entropy: 0.7924812503605781 perplexity: 1.7320508075688774
['<s>', 'Sam', 'I', 'am', '</s>'] cross entropy: 1.042481250360578 perplexity: 2.0597671439071177
['<s>', 'I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '</s>'] cross entropy: 0.24110277793803472 perplexity: 1.1818957424692271
bigram corpus cross entropy 0.5593985296662904
bigram corpus perplexity 1.4736547115524326


# Exercises

# Exercise 1. Defining Vocabulary, using OOV word token and building a unigram MLE model from a bigger corpus

Write code to read in the file `switchboard_lm_train.txt` which has preprocessed text on each line.

You will populate a unigram language model based on that data for an MLE estimation using a `Counter` using this data (see Example 1 above as to how this is done for a smaller dataset).

Before you do this, you have to define a vocabulary of words using the training data, which you will keep the same for all of the following exercises. In these exercises, the vocabulary is defined by using a Minimum Document Frequency of 2 in the training data. That means the vocab should only contain words which occur at least twice in the training data.

Any words not in the vocabulary in the training, heldout and testing data must be replaced with an out-of-vocab symbol `<unk/>` before processing them.
 
Using this model, calculate the perplexity of the ENTIRE test corpus `switchboard_lm_test.txt`- again, remember to replace words unknown by the model with `<unk/>` before getting these measures. See Example 1 as to how this is done for a unigram model on the smaller example dataset.

In [12]:
##############################################
# first get the vocab

corpus = open("switchboard_lm_train.txt")
initial_unigrams = Counter()
for line in corpus:
    words = tokenize_sentence(line, 1)
    for w in words:
        initial_unigrams[w]+=1
corpus.close()

vocab = []
for w,v in initial_unigrams.items():
    if v>=2:
        vocab.append(w)
print(len(vocab))

12729


In [13]:
##############################################
# next do the replacement of out-of-vocab (OOV) words with <unk/> in training sentences

train_sentences = []
corpus = open("switchboard_lm_train.txt")
unknown_words = 0
for line in corpus:
    words = tokenize_sentence(line, 1)
    final_words = []
    for w in words[:-1]:
        word = w
        if w not in vocab:
            word = "<unk/>"
            unknown_words += 1
        final_words.append(word)
    train_sentences.append(" ".join(final_words))
print("unknown words in training data", unknown_words)
corpus.close()

unknown words in training data 7898


In [14]:
unigrams = Counter()
for sent in train_sentences:
    words = tokenize_sentence(sent, 1)
    for word in words:
        unigrams[word]+=1
unigram_total = sum(unigrams.values())

In [15]:
print(unigram_total)

1306313


In [16]:
##############################################
# next do the replacement of out-of-vocab (OOV) words with <unk/> in test sentences

test_sentences = []
corpus = open("switchboard_lm_test.txt")
unknown_words = 0
for line in corpus:
    words = tokenize_sentence(line, 1)
    final_words = []
    for w in words[:-1]:
        word = w
        if w not in vocab:
            word = "<unk/>"
            unknown_words +=1
        final_words.append(word)
    test_sentences.append(" ".join(final_words))
print("unknown words in test data", unknown_words)
corpus.close()

unknown words in test data 628


In [17]:
N = 0 # total number of words
s = 0 # neg sum of log probs
for sent in test_sentences:
    words = tokenize_sentence(sent, 1)
    for word in words:
        N += 1
        num = unigrams[word]   # numerator is count of unigram
        denom = unigram_total  # denominator is overall length of training data
        prob = num / denom
        log_prob = log(prob, 2)
        s += -log_prob

perplexity = 2 ** (s/N)
print("(Q1) cross entropy", s/N)
print("(Q1) perplexity", perplexity)

(Q1) cross entropy 8.287702715147693
(Q1) perplexity 312.4979066155026



# Exercise 2. Bigram model with add-one smoothing

Change your method for reading in and training a language model from Exercise 1 so it works for bigrams. Use the same methods for identifying and replacing out-of-vocabulary words as you did in Exercise 1 (i.e. use the same vocabulary).

However, in testing, rather than just using the raw counts for implementing MLE probabilities you should implement add-one smoothing (see the lecture notes and Jurafsky & Martin Chapter 3/Manning and Schuetze Chapter 6). Remember this involves using the vocabulary size in the denominator of the formula.

Obtain the perplexity score on the test data as above for this smoothed bigram model.

In [18]:
# get the counts for the bigram model
bigrams = Counter()  # counts all the bigrams
bigram_context = Counter() # counts unigrams, but the previous word only (so includes the start symbol, not the end)
delta = 1  # delta is order - 1
vocab = vocab + ["<s>"] # add the start symbol for bigrams

for sent in train_sentences:
    words = tokenize_sentence(sent, 2)
    for i in range(delta, len(words)):
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        bigrams[glue_tokens(ngram, 2)] +=1
        bigram_context[glue_tokens(context, 1)] += 1

In [19]:
# get the test results
# get the entropy/perplexity for plus 1 smoothing on the TEST data (Q2)
# for best result, test it on the test data
N = 0 # total number of words
s = 0 # mass of neglogprob (entropy)
for sent in test_sentences:
    words = tokenize_sentence(sent, 2)
    for i in range(delta, len(words)):
        N += 1
        context = words[i-delta:i]
        target = words[i]
        ngram = context + [target]
        num = bigrams[glue_tokens(ngram, 2)] + 1 # add 1 to the numerator
        denom = bigram_context[glue_tokens(context, 1)] + len(bigram_context.keys()) # add V to the denominator
        prob = num/denom
        log_prob = log(prob, 2)
        s += (-log_prob)
print("add-1 smoothing")
perplexity = 2 ** (s/N)
print("cross entropy", s/N)
print("perplexity", perplexity)

add-1 smoothing
cross entropy 8.248721413034907
perplexity 304.1673339728869


# Exercise 3. Bigram model with general additive (Lidstone) add-k smoothing

Modify your code from Exercise 2 such that it generalizes beyond adding 1 to all counts, but can add differing amounts k of mass instead, to implement general additive add-k smoothing.

On the HELDOUT corpus `switchboard_lm_heldout.txt` experiment with different values of k (e.g. 0.2, 0.4, 0.6, 0.8, though try others if you can) and report the perplexity scores for these different values in a comment. You could also use an optimization algorithm from scipy.optimize to search this efficiently.

Once you find the value which gives you the lowest perplexity on the heldout data, use this model to get the perplexity of the test data once and report the scores in a comment. You should be able to get better (lower) perplexity scores than plus-1 smoothing, however make sure the vocabulary used is the same, else it is not a fair comparison.

In [20]:
##############################################
# next do the replacement of out-of-vocab (OOV) words with <unk/> in heldout sentences
heldout_sentences = []
unknown_words = 0
corpus = open("switchboard_lm_heldout.txt")
for line in corpus:
    words = tokenize_sentence(line, 1)
    final_words = []
    for w in words[:-1]:
        word = w
        if w not in vocab:
            word = "<unk/>"
            unknown_words += 1
        final_words.append(word)
    heldout_sentences.append(" ".join(final_words))
print("unknown words in heldout set", unknown_words)
corpus.close()

unknown words in heldout set 634


In [21]:
# test a range of k for add-k smoothing

results = {}
for raw_weight in [i for i in range(1, 11)] + [100, 200, 400, 600, 800]:  # test 0.001..0.01 and 0.1, 0.2, 0.4, etc.
    weight = raw_weight/1000

    N = 0 # total number of words
    s = 0 # mass of neglogprob (entropy)
    for sent in heldout_sentences:
        words = tokenize_sentence(sent, 2)
        for i in range(delta, len(words)):
            N += 1
            context = words[i-delta:i]
            target = words[i]
            ngram = context + [target]
            num = bigrams[glue_tokens(ngram, 2)] + weight  # add k to numerator
            denom = bigram_context[glue_tokens(context, 1)] + (weight * len(bigram_context.keys()))
            prob = num / denom
            log_prob = log(prob, 2)
            s += (-log_prob)

    perplexity = 2 ** (s/N)
    print("add-" + str(weight), s/N, perplexity)
    results[weight] = (s/N, perplexity)

add-0.001 6.914173989499943 120.60734746883352
add-0.002 6.859854832769601 116.15076394061568
add-0.003 6.838662047978613 114.45701267621813
add-0.004 6.829189781774097 113.70798679465172
add-0.005 6.825360477621457 113.40657513110997
add-0.006 6.824686853265728 113.35363559997062
add-0.007 6.825941774864039 113.45227863068037
add-0.008 6.8284452414699635 113.64931996979587
add-0.009 6.8317891784575275 113.91304648576887
add-0.01 6.835714045927112 114.22337008930648
add-0.1 7.199013500070242 146.93288389670292
add-0.2 7.439684502618534 173.60738423639475
add-0.4 7.753563156324377 215.80181019748358
add-0.6 7.971728174404209 251.0321223111287
add-0.8 8.141488581198528 282.37891962673933


In [22]:
#print("(Q2) weight 1 (add-1 smoothing) cross-entropy, perplexity:", results[1.0])
print("(Q3) weight 0.8 cross-entropy, perplexity:", results[0.8])
print("(Q3) weight 0.6 cross-entropy, perplexity:", results[0.6])
print("(Q3) weight 0.4 cross-entropy, perplexity:", results[0.4])
print("(Q3) weight 0.2 cross-entropy, perplexity:", results[0.2])
print("(Q3) weight 0.1 cross-entropy, perplexity:", results[0.1])
print("(Q3) weight 0.01 cross-entropy, perplexity:", results[0.01])
print("(Q3) lowest cross-entropy, perplexity model", min(results.items(), key=lambda x:x[1]))

(Q3) weight 0.8 cross-entropy, perplexity: (8.141488581198528, 282.37891962673933)
(Q3) weight 0.6 cross-entropy, perplexity: (7.971728174404209, 251.0321223111287)
(Q3) weight 0.4 cross-entropy, perplexity: (7.753563156324377, 215.80181019748358)
(Q3) weight 0.2 cross-entropy, perplexity: (7.439684502618534, 173.60738423639475)
(Q3) weight 0.1 cross-entropy, perplexity: (7.199013500070242, 146.93288389670292)
(Q3) weight 0.01 cross-entropy, perplexity: (6.835714045927112, 114.22337008930648)
(Q3) lowest cross-entropy, perplexity model (0.006, (6.824686853265728, 113.35363559997062))


In [23]:
# aroound 0.006 seems to be the best (lowest cross-entropy/peplexity)
# more principled way of optimizing, even to 4 decimal places:
def perplexity_of_add_k_bigram(weight, sentences):
    N = 0 # total number of words
    s = 0 # mass of neglogprob (entropy)
    for sent in sentences:
        words = tokenize_sentence(sent, 2)
        for i in range(delta, len(words)):
            N += 1
            context = words[i-delta:i]
            target = words[i]
            ngram = context + [target]
            num = bigrams[glue_tokens(ngram, 2)] + weight  # add k to numerator
            denom = bigram_context[glue_tokens(context, 1)] + (weight * len(bigram_context.keys()))
            prob = num / denom
            log_prob = log(prob, 2)
            s += (-log_prob)

    perplexity = 2 ** (s/N)
    print("add-" + str(weight), s/N, perplexity)
    return perplexity


In [24]:
# use scipy optimize to get even better performance
from scipy import optimize
best = optimize.minimize(
    perplexity_of_add_k_bigram,
    0.01,    # first argument that to be optimized
    args=(heldout_sentences),  # other arguments to the function
    method='Nelder-Mead', # use nelder mead to find minima
    tol=0.0001, # to this degree of error
    options={'disp': True}
)
best_k = best.x[0]
print("best k", best_k)

add-[0.01] 6.835714045927112 114.22337008930648
add-[0.0105] 6.837838042764665 114.39165843026665
add-[0.0095] 6.833691336774233 114.0633371220369
add-[0.009] 6.8317891784575275 113.91304648576887
add-[0.008] 6.8284452414699635 113.64931996979587
add-[0.007] 6.825941774864039 113.45227863068037
add-[0.005] 6.825360477621457 113.40657513110997
add-[0.003] 6.838662047978613 114.45701267621813
add-[0.003] 6.838662047978613 114.45701267621813
add-[0.006] 6.824686853265728 113.35363559997062
add-[0.007] 6.825941774864039 113.45227863068037
add-[0.0055] 6.824723732385953 113.35653325724958
add-[0.0065] 6.825124246723737 113.38800714337503
add-[0.00575] 6.824639034502369 113.3498785059953
add-[0.0055] 6.824723732385953 113.35653325724958
add-[0.005875] 6.8246473305047255 113.35053030941613
add-[0.005625] 6.824663831647001 113.35182678848572
add-[0.0058125] 6.824639165729114 113.34988881623819
Optimization terminated successfully.
         Current function value: 113.349879
         Iterations

In [25]:
# get test result
print("best-k perplexity on test", perplexity_of_add_k_bigram(best_k, test_sentences))

add-0.0057499999999999895 6.806850670324508 111.96086052571216
best-k perplexity on test 111.96086052571216



# Exercise 4. Ngram models with Kneser-Ney smoothing

Kneser-Ney smoothing is a state-of-the-art technique for smoothing n-gram models.

The algorithm is quite complicated, and is implemented for you below for training/getting the appropriate counts  on the training data (`count_ngrams_interpolated_kneser_ney()`). The training process is implemented below for a trigram model, but without doing the appropriate out-of-vocab word replacement as you've done above, using exactly the same vocabulary.

The application at test time is done with the method `kneser_ney_ngram_prob()` using the trained Counters, which gives the probability of the model applied to an ngram of a given order, with a given discount.

Try to follow how the training works and how the application of the model to ngrams works, and refer to both Section 3.5 in Jurafsky and Martin and the below article on QM plus (pages 7-8 particularly):

"A Bit of Progress in Language Modeling" (2001) - Joshua T. Goodman

In this exercise, you will first modify the training part of the code so it does the replacement of out-of-vocab words as you did in the previous exercises. You do not need to modify the methods below.

On the HELDOUT corpus experiment with different orders from trigram upwards (try 3, 4 and 5) and different discount values (e.g. 0.2, 0.4, 0.6, 0.8, though try others if you can) and report the perplexity scores for these different values in a comment. You could also use an optimization algorithm from scipy.optimize to search the different n values and discount values efficiently. 

Once you find the order and discount values which gives you the lowest perplexity on the heldout data, use this model to get the perplexity of the TEST data once and report the scores in a comment. You should be able to beat (get a lower) perplexity for this compared to excercises 1-3, though make sure you are always using the same vocabulary, else it is not a fair comparison.

In [26]:
# Kneser-Ney smoothing
order = 3  # use a trigram
delta = order - 1
discount = 0.8  # use a discount of 0.8


# initialize the maps and counts
unigram_denominator = 0
ngram_numerator_map = Counter() 
ngram_denominator_map = Counter() 
ngram_non_zero_map = Counter()


def count_ngrams_interpolated_kneser_ney(tokens,
                                   order,
                                   ngram_numerator_map,
                                   ngram_denominator_map,
                                   ngram_non_zero_map,
                                   unigram_denominator):
    """Function used in n-gram language model training
    to count the n-grams in tokens and also record the
    lower order non -ero counts necessary for interpolated Kneser-Ney
    smoothing.
    
    Taken from Goodman 2001 and generalized to arbitrary orders"""
    for i in range(order-1,len(tokens)): # tokens should have a prefix of order - 1
        #print(i)
        for d in range(order,0,-1): #go through all the different 'n's backwards
            if d == 1:
                unigram_denominator += 1   # it has been in a context
                ngram_numerator_map[glue_tokens(tokens[i],d)] += 1
            else:
                den_key = glue_tokens(tokens[i-(d-1) : i], d)
                num_key = glue_tokens(tokens[i-(d-1) : i+1], d)
                # increment the number of times the denominator/context has been seen by 1
                ngram_denominator_map[den_key] += 1
                # we store the numerator value to check if it's 0
                tmp = ngram_numerator_map[num_key]
                # we increment the number of times it's been observed as a numerator
                ngram_numerator_map[num_key] += 1
                # this incrementation of the n-gram count if d < order
                # will only happen if the ngram tested at d+1 was unique
                if tmp == 0:
                    # if this is the first time we see this ngram
                    # increment number of types for which its context
                    # has been used as a context for any continuation
                    ngram_non_zero_map[den_key] += 1
                else:
                    break 
                    # if the ngram has already been seen
                    # we don't go down to any lower order models
                    # because they will have already been counted as types
    # return the updated counts and maps
    return ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator


In [27]:
def kneser_ney_ngram_prob(ngram, discount, order):
    """KN smoothed ngram probability from Goodman 2001.
    This is run at test time to calculate the probability
    of a given n-gram or a given order with a given discount.
    
    ngram :: list of strings, the ngram
    discount :: float, the discount used
    order :: int, order of the model
    """
    # First, calculate the unigram continuation prob of the last token
    # If we've never seen it at all, it will 
    # have no probability as a numerator
    uni_num = ngram_numerator_map.get(glue_tokens(ngram[-1], 1)) # number of bigrams the word is a continuation for
    if not uni_num: # if no value found in dict, make it 0
        uni_num = 0
    # unigram_denominator is the number of different bigram types (not tokens)
    probability = previous_prob = uni_num / unigram_denominator
    
    # Check: Given <unk/> should have been used in place of unknown words before passing
    # to this method, probability should be non-zero
    if probability == 0.0:
        print("0 prob for unigram!")
        print(glue_tokens(ngram[-1], 1))
        print(ngram)
        print(ngram_numerator_map.get(glue_tokens(ngram[-1], 1)))
        print(unigram_denominator)
        raise Exception

    # Compute the higher order probs (from 2/bi-gram upwards) and interpolate them
    for d in range(2,order+1):
        # Get the context count for the denominator:
        # When d = order (n) this is the number of times it's observed in the corpus (tokens)
        # When d < order (n) this is the number of different continuation types (not tokens) seen with it as its prefix
        ngram_denom = ngram_denominator_map.get(glue_tokens(ngram[-(d):-1], d))
        if not ngram_denom: # if no value found in dict, make it 0
            ngram_denom = 0
        if ngram_denom != 0:
            # Get the ngram count for the numerator
            # When d = order (n) this is the number of times it's observed in the corpus (tokens)
            # When d < order (n) this is the number of types of unique ngram for n=d+1 it is a continuation for (types)
            ngram_num = ngram_numerator_map.get(glue_tokens(ngram[-(d):], d))
            if not ngram_num: # if no value found in dict, make it 0
                ngram_num = 0
            if ngram_num != 0:
                # calculate the prob with fixed discount
                current_prob = (ngram_num - discount) / ngram_denom
            else:
                current_prob = 0.0
            # get the number of word types that can follow the context
            # (number of times normalised discount has been applied):
            nonzero = ngram_non_zero_map.get(glue_tokens(ngram[-(d):-1], d))
            if not nonzero: # if no value found in dict, make it 0
                nonzero = 0
            # get the lambda for this context
            lambda_context = discount / ngram_denom * nonzero
            # interpolate with previous probability of lower orders calculated so far
            current_prob += lambda_context * previous_prob
            previous_prob = current_prob
            probability = current_prob
        else:
            #if this context (e.g. bigram context for trigrams) has never been seen, 
            #then we can only use the last order with a probability (e.g. unigram)
            #and halt
            probability = previous_prob
            break
    return probability

In [28]:
# search over n == 3, 4, 5
# try discounts from 0.60 to 1.00 in 0.01 increments
results  = {}
for order in range(3,6):
    # reset each time
    delta = order -1
    unigram_denominator = 0
    ngram_numerator_map = Counter() 
    ngram_denominator_map = Counter() 
    ngram_non_zero_map = Counter()

    # for each n- get the counts from training
    for sent in train_sentences:
        tokens = tokenize_sentence(sent, order)
        # correct use of kneser-ney probability function:
        ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator =\
                count_ngrams_interpolated_kneser_ney(tokens, order,
                                               ngram_numerator_map, ngram_denominator_map,
                                      ngram_non_zero_map, unigram_denominator)

    for raw_discount in range(60, 101):
        discount = raw_discount / 100
        #print("*" * 30)
        #print("order", order)
        print("discount", discount)
        
        N = 0
        s = 0
        unknown_words = 0
        for sent in heldout_sentences:
            tokens = tokenize_sentence(sent, order)
            for i in range(order-1, len(tokens)):
                N +=1
                ngram = tokens[i-delta:i+1]
                kn_prob = kneser_ney_ngram_prob(ngram, discount, order)
                s+= (-log(kn_prob, 2))
        perplexity = 2 ** (s/N)
        print(order, discount, s/N, perplexity)
        results[(order, discount)] = (s/N, perplexity)

discount 0.6
3 0.6 6.241801452580393 75.67796830136515
discount 0.61
3 0.61 6.2357080364676465 75.35900630992558
discount 0.62
3 0.62 6.229811114113146 75.05160992150178
discount 0.63
3 0.63 6.224108202646403 74.75551973583148
discount 0.64
3 0.64 6.218597182550095 74.47050225716646
discount 0.65
3 0.65 6.213276298293761 74.19634930355728
discount 0.66
3 0.66 6.208144161801208 73.93287761070887
discount 0.67
3 0.67 6.203199758956959 73.67992863536205
discount 0.68
3 0.68 6.198442459413827 73.43736856662021
discount 0.69
3 0.69 6.1938720300411445 73.20508855809508
discount 0.7
3 0.7 6.189488652456602 72.983005199455
discount 0.71
3 0.71 6.185292945187324 72.77106125157445
discount 0.72
3 0.72 6.181285991158633 72.56922667760851
discount 0.73
3 0.73 6.177469371385676 72.37750001161491
discount 0.74
3 0.74 6.173845205956239 72.19591011748707
discount 0.75
3 0.75 6.170416203700359 72.02451840685906
discount 0.76
3 0.76 6.167185722296468 71.86342160294313
discount 0.77
3 0.77 6.164157841063

In [29]:
results

{(3, 0.6): (6.241801452580393, 75.67796830136515),
 (3, 0.61): (6.2357080364676465, 75.35900630992558),
 (3, 0.62): (6.229811114113146, 75.05160992150178),
 (3, 0.63): (6.224108202646403, 74.75551973583148),
 (3, 0.64): (6.218597182550095, 74.47050225716646),
 (3, 0.65): (6.213276298293761, 74.19634930355728),
 (3, 0.66): (6.208144161801208, 73.93287761070887),
 (3, 0.67): (6.203199758956959, 73.67992863536205),
 (3, 0.68): (6.198442459413827, 73.43736856662021),
 (3, 0.69): (6.1938720300411445, 73.20508855809508),
 (3, 0.7): (6.189488652456602, 72.983005199455),
 (3, 0.71): (6.185292945187324, 72.77106125157445),
 (3, 0.72): (6.181285991158633, 72.56922667760851),
 (3, 0.73): (6.177469371385676, 72.37750001161491),
 (3, 0.74): (6.173845205956239, 72.19591011748707),
 (3, 0.75): (6.170416203700359, 72.02451840685906),
 (3, 0.76): (6.167185722296468, 71.86342160294313),
 (3, 0.77): (6.164157841063159, 71.71275516295331),
 (3, 0.78): (6.161337449316755, 71.57269750422354),
 (3, 0.79): (6

In [30]:
# find the best discounts for each n-gram and overall
print("(Q4) lowest cross-entropy/perplexity model", min(results.items(), key=lambda x:x[1][1]))
for n in [3, 4, 5]:
    ngram = filter(lambda x:x[0][0]==n, results.items())
    print("(Q4) lowest cross-entropy/perplexity {}-gram model".format(n), min(ngram, key=lambda x:x[1]))

(Q4) lowest cross-entropy/perplexity model ((4, 0.92), (6.123593031944027, 69.72446391965357))
(Q4) lowest cross-entropy/perplexity 3-gram model ((3, 0.88), (6.146519960360805, 70.84135786456216))
(Q4) lowest cross-entropy/perplexity 4-gram model ((4, 0.92), (6.123593031944027, 69.72446391965357))
(Q4) lowest cross-entropy/perplexity 5-gram model ((5, 0.93), (6.126593059451137, 69.86960405009052))


In [31]:
#try optimization for each order and get the best value

def perplexity_kn_ngram(discount, order, sentences):
    print(discount, order)
    
    #print("*" * 30)
    #print("order", order)
    print("discount", discount)

    N = 0
    s = 0
    unknown_words = 0
    for sent in sentences:
        tokens = tokenize_sentence(sent, order)
        for i in range(order-1, len(tokens)):
            N +=1
            ngram = tokens[i-delta:i+1]
            kn_prob = kneser_ney_ngram_prob(ngram, discount, order)
            s+= (-log(kn_prob, 2))
    perplexity = 2 ** (s/N)
    print(order, discount, s/N, perplexity)
    return perplexity

In [ ]:
overall_best_perplex = 10000
best_order_k = (0,0)
for order in range(3,6):
    # reset each time
    delta = order -1
    unigram_denominator = 0
    ngram_numerator_map = Counter() 
    ngram_denominator_map = Counter() 
    ngram_non_zero_map = Counter()

    # for each n- get the counts from training
    for sent in train_sentences:
        tokens = tokenize_sentence(sent, order)
        # correct use of kneser-ney probability function:
        ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator =\
                count_ngrams_interpolated_kneser_ney(tokens, order,
                                               ngram_numerator_map, ngram_denominator_map,
                                      ngram_non_zero_map, unigram_denominator)

    best = optimize.minimize(
        perplexity_kn_ngram,
        0.8,    # first argument to be optimized
        args=(order, heldout_sentences),  # other arguments to the function
        method='Nelder-Mead', # use nelder mead to find minima
        tol=0.0001, # to this degree of error
        options={'disp': True}
    )
    k = best.x[0]
    if best.fun < overall_best_perplex:
        overall_best_perplex = best.fun
        best_order_k = (order, k)
    print("best k for n=",order,":", k)
print("Overall order, k", best_order_k, "perplexity:", overall_best_perplex)

[0.8] 3
discount [0.8]
3 [0.8] 6.156343411773763 71.32536955933497
[0.84] 3
discount [0.84]
3 [0.84] 6.149181348370975 70.97216190958851
[0.88] 3
discount [0.88]
3 [0.88] 6.146519960360803 70.84135786456207
[0.92] 3
discount [0.92]
3 [0.92] 6.15005997784283 71.01539850770094
[0.92] 3
discount [0.92]
3 [0.92] 6.15005997784283 71.01539850770094
[0.86] 3
discount [0.86]
3 [0.86] 6.147212094770381 70.87535223163498
[0.9] 3
discount [0.9]
3 [0.9] 6.147345459258133 70.88190433840678
[0.87] 3
discount [0.87]
3 [0.87] 6.146693603770601 70.84988487470652
[0.89] 3
discount [0.89]
3 [0.89] 6.146723787608154 70.85136720030286
[0.875] 3
discount [0.875]
3 [0.875] 6.146561794042411 70.84341207400546
[0.885] 3
discount [0.885]
3 [0.885] 6.146572374800697 70.84393164310829
[0.8775] 3
discount [0.8775]
3 [0.8775] 6.146529373623677 70.84182009010561
[0.8825] 3
discount [0.8825]
3 [0.8825] 6.146534101681184 70.84205225611345
[0.87875] 3
discount [0.87875]
3 [0.87875] 6.14652175755727 70.84144611323124
[0

In [ ]:
best_order_k

In [ ]:
# use the best order and discount on test data
order = best_order_k[0]
discount = best_order_k[1]
    
delta = order -1
unigram_denominator = 0
ngram_numerator_map = Counter() 
ngram_denominator_map = Counter() 
ngram_non_zero_map = Counter()

# for each n- get the counts from training
for sent in train_sentences:
    tokens = tokenize_sentence(sent, order)
    # correct use of kneser-ney probability function:
    ngram_numerator_map, ngram_denominator_map, ngram_non_zero_map, unigram_denominator =\
            count_ngrams_interpolated_kneser_ney(tokens, order,
                                           ngram_numerator_map, ngram_denominator_map,
                                  ngram_non_zero_map, unigram_denominator)

print("perplexity on test", perplexity_kn_ngram(discount, order, test_sentences))